# History

### Ver2
* VIF 분석을 통해 피처 선택을 수행하고 테스트합니다.
* VIF 분석 결과 제거할 피처가 없어 permutation_importance 함수를 사용 후 제거할 피쳐를 선택합니다.
* Perform feature selection through VIF analysis and test it.
* After conducting the VIF analysis and finding no features to remove, I will now use the permutation_importance function to determine which features to remove.

### Ver3
* RNF와 Type컬럼을 제거한 데이터에 대해서 모델을 학습하고 튜닝합니다.
* Train and fine-tune the model using the data excluding the 'RNF' and 'Type' columns.

### Ver5
* 버전 4는 기각되고 DL 모델의 구조를 변경하고 다시 튜닝을 시도합니다.
* Version 4 is rejected, and we will proceed to alter the structure of the deep learning model and attempt tuning once again.

In [1]:
# !pip install git+https://github.com/tooha289/DataAnalysisLibrary.git

# 사용 모듈 Import

In [45]:
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from DataAnalysis import eda
from DataAnalysis import feature_engineering

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.model_selection import StratifiedKFold

from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.optimizers import Adam

import optuna

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.metrics import roc_auc_score

# 0. Check GPU

In [3]:
import tensorflow as tf

# GPU 디바이스 목록 가져오기
gpu_devices = tf.config.experimental.list_physical_devices('GPU')

# GPU 사용 가능한 경우
if gpu_devices:
    for device in gpu_devices:
        print("GPU 사용 가능:", device)
else:
    print("GPU 사용 불가능")

GPU 사용 가능: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


# 1. Load & Check Data

In [4]:
df = pd.read_csv('../data/train.csv')
df

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,0,L50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,1,M20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,2,L49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0,0
3,3,L53355,L,301.0,310.9,1524,44.3,197,0,0,0,0,0,0
4,4,M24050,M,298.0,309.0,1641,35.4,34,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136424,136424,M22284,M,300.1,311.4,1530,37.5,210,0,0,0,0,0,0
136425,136425,H38017,H,297.5,308.5,1447,49.1,2,0,0,0,0,0,0
136426,136426,L54690,L,300.5,311.8,1524,38.5,214,0,0,0,0,0,0
136427,136427,L53876,L,301.7,310.9,1447,46.3,42,0,0,0,0,0,0


In [5]:
test_df = pd.read_csv('../data/test.csv')
test_df

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF
0,136429,L50896,L,302.3,311.5,1499,38.0,60,0,0,0,0,0
1,136430,L53866,L,301.7,311.0,1713,28.8,17,0,0,0,0,0
2,136431,L50498,L,301.3,310.4,1525,37.7,96,0,0,0,0,0
3,136432,M21232,M,300.1,309.6,1479,47.6,5,0,0,0,0,0
4,136433,M19751,M,303.4,312.3,1515,41.3,114,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90949,227378,L51130,L,302.3,311.4,1484,40.4,15,0,0,0,0,0
90950,227379,L47783,L,297.9,309.8,1542,33.8,31,0,0,0,0,0
90951,227380,L48097,L,295.6,306.2,1501,41.4,187,0,0,0,0,0
90952,227381,L48969,L,298.1,307.8,1534,40.3,69,0,0,0,0,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136429 entries, 0 to 136428
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       136429 non-null  int64  
 1   Product ID               136429 non-null  object 
 2   Type                     136429 non-null  object 
 3   Air temperature [K]      136429 non-null  float64
 4   Process temperature [K]  136429 non-null  float64
 5   Rotational speed [rpm]   136429 non-null  int64  
 6   Torque [Nm]              136429 non-null  float64
 7   Tool wear [min]          136429 non-null  int64  
 8   Machine failure          136429 non-null  int64  
 9   TWF                      136429 non-null  int64  
 10  HDF                      136429 non-null  int64  
 11  PWF                      136429 non-null  int64  
 12  OSF                      136429 non-null  int64  
 13  RNF                      136429 non-null  int64  
dtypes: f

In [7]:
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,136429.0,NaN,NaN,NaN,68214.0,39383.804275,0.0,34107.0,68214.0,102321.0,136428.0
Product ID,136429,9976,L53257,139,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Type,136429,3,L,95354,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Air temperature [K],136429.0,NaN,NaN,NaN,299.862776,1.862247,295.3,298.3,300.0,301.2,304.4
Process temperature [K],136429.0,NaN,NaN,NaN,309.94107,1.385173,305.8,308.7,310.0,310.9,313.8
Rotational speed [rpm],136429.0,NaN,NaN,NaN,1520.33111,138.736632,1181.0,1432.0,1493.0,1580.0,2886.0
Torque [Nm],136429.0,NaN,NaN,NaN,40.348643,8.502229,3.8,34.6,40.4,46.1,76.6
Tool wear [min],136429.0,NaN,NaN,NaN,104.408901,63.96504,0.0,48.0,106.0,159.0,253.0
Machine failure,136429.0,NaN,NaN,NaN,0.015744,0.124486,0.0,0.0,0.0,0.0,1.0
TWF,136429.0,NaN,NaN,NaN,0.001554,0.039389,0.0,0.0,0.0,0.0,1.0


In [8]:
df.isnull().sum()

id                         0
Product ID                 0
Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
Machine failure            0
TWF                        0
HDF                        0
PWF                        0
OSF                        0
RNF                        0
dtype: int64

# 2. Separate columns

In [9]:
feature_cols = df.columns.difference(['id', 'Product ID', 'Machine failure'])
numeric_cols = df.columns.difference(['id', 'Product ID', 'Machine failure', 'Type'])
continuos_cols = feature_cols.difference(["TWF", "HDF", "PWF", "OSF", "RNF", "Type"])
discrete_cols = feature_cols.difference(continuos_cols)

# 3. FeatureEngineering

## 3.1 Preprocessing

In [10]:
dfp = feature_engineering.DataFramePreprocessor()
le = LabelEncoder()
stds = StandardScaler()

### 3.1.1 Train set

In [11]:
x_data = df.copy()
x_data = x_data.drop(labels=['id', 'Product ID'], axis=1)
x_data, _ = dfp.fit_transform_multiple_transformer(x_data, [le, stds], [["Type"], continuos_cols])
x_data = pd.concat([x_data, df[discrete_cols.difference(['Type'])]], axis=1)

x_data[discrete_cols] = x_data[discrete_cols].astype("int8")
x_data

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Tool wear [min],Torque [Nm],HDF,OSF,PWF,RNF,TWF
0,1,0.395880,-0.246230,0.545416,0.556417,-0.499711,0,0,0,0,0
1,2,1.469856,1.558605,1.720308,1.494433,-1.323028,0,0,0,0,0
2,1,-0.302204,-1.040358,2.051873,-1.241447,-1.628831,0,0,0,0,0
3,1,0.610675,0.692284,0.026445,1.447532,0.464745,0,0,0,0,0
4,2,-1.000288,-0.679391,0.869773,-1.100744,-0.582043,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
136424,2,0.127386,1.053251,0.069693,1.650769,-0.335048,0,0,0,0,0
136425,0,-1.268781,-1.040358,-0.528565,-1.601020,1.029305,0,0,0,0,0
136426,1,0.342182,1.342025,0.026445,1.713303,-0.217431,0,0,0,0,0
136427,1,0.986567,0.692284,-0.528565,-0.975676,0.699979,0,0,0,0,0


In [12]:
x_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136429 entries, 0 to 136428
Data columns (total 11 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Type                     136429 non-null  int8   
 1   Air temperature [K]      136429 non-null  float64
 2   Process temperature [K]  136429 non-null  float64
 3   Rotational speed [rpm]   136429 non-null  float64
 4   Tool wear [min]          136429 non-null  float64
 5   Torque [Nm]              136429 non-null  float64
 6   HDF                      136429 non-null  int8   
 7   OSF                      136429 non-null  int8   
 8   PWF                      136429 non-null  int8   
 9   RNF                      136429 non-null  int8   
 10  TWF                      136429 non-null  int8   
dtypes: float64(5), int8(6)
memory usage: 6.0 MB


In [13]:
y_data = df.copy().pop('Machine failure')
y_data = y_data.astype("int8")
y_data

0         0
1         0
2         0
3         0
4         0
         ..
136424    0
136425    0
136426    0
136427    0
136428    0
Name: Machine failure, Length: 136429, dtype: int8

### 3.1.2 Test set

In [14]:
x_test = test_df.copy()
x_test = x_test.drop(labels=['id', 'Product ID'], axis=1)
x_test, _ = dfp.fit_transform_multiple_transformer(x_test, [le, stds], [["Type"], continuos_cols])
x_test = pd.concat([x_test, test_df[discrete_cols.difference(['Type'])]], axis=1)

x_test[discrete_cols] = x_test[discrete_cols].astype("int8")
x_test

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Tool wear [min],Torque [Nm],HDF,OSF,PWF,RNF,TWF
0,1,1.313830,1.126570,-0.153806,-0.693494,-0.274579,0,0,0,0,0
1,1,0.990824,0.765635,1.375097,-1.366729,-1.356341,0,0,0,0,0
2,1,0.775487,0.332512,0.031949,-0.129855,-0.309854,0,0,0,0,0
3,2,0.129475,-0.244985,-0.296694,-1.554608,0.854217,0,0,0,0,0
4,2,1.906008,1.704068,-0.039496,0.151964,0.113445,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
90949,1,1.313830,1.054383,-0.260972,-1.398042,0.007620,0,0,0,0,0
90950,1,-1.054879,-0.100611,0.153403,-1.147536,-0.768427,0,0,0,0,0
90951,1,-2.293068,-2.699348,-0.139517,1.294897,0.125203,0,0,0,0,0
90952,1,-0.947211,-1.544354,0.096248,-0.552584,-0.004138,0,0,0,0,0


In [15]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90954 entries, 0 to 90953
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Type                     90954 non-null  int8   
 1   Air temperature [K]      90954 non-null  float64
 2   Process temperature [K]  90954 non-null  float64
 3   Rotational speed [rpm]   90954 non-null  float64
 4   Tool wear [min]          90954 non-null  float64
 5   Torque [Nm]              90954 non-null  float64
 6   HDF                      90954 non-null  int8   
 7   OSF                      90954 non-null  int8   
 8   PWF                      90954 non-null  int8   
 9   RNF                      90954 non-null  int8   
 10  TWF                      90954 non-null  int8   
dtypes: float64(5), int8(6)
memory usage: 4.0 MB


## 3.2 Feature selecting

### 3.2.1 VIF analysis

In [16]:
fs = feature_engineering.FeatureSelector()

In [17]:
formula_cols = pd.Series(feature_cols).apply(lambda col: f"Q('{col}')")
formula_cols

0         Q('Air temperature [K]')
1                         Q('HDF')
2                         Q('OSF')
3                         Q('PWF')
4     Q('Process temperature [K]')
5                         Q('RNF')
6      Q('Rotational speed [rpm]')
7                         Q('TWF')
8             Q('Tool wear [min]')
9                 Q('Torque [Nm]')
10                       Q('Type')
dtype: object

In [18]:
formula = "Q('Machine failure')~" + "+".join(formula_cols)
formula+="-1"
formula

"Q('Machine failure')~Q('Air temperature [K]')+Q('HDF')+Q('OSF')+Q('PWF')+Q('Process temperature [K]')+Q('RNF')+Q('Rotational speed [rpm]')+Q('TWF')+Q('Tool wear [min]')+Q('Torque [Nm]')+Q('Type')-1"

In [19]:
data = pd.concat([x_data, y_data], axis=1)

In [20]:
fs.get_vif_dataframe(formula, data)

,features,VIF Factor
0,Q('Air temperature [K]'),3.806145
4,Q('Process temperature [K]'),3.772470
9,Q('Torque [Nm]'),2.629339
6,Q('Rotational speed [rpm]'),2.608535
1,Q('HDF'),1.038264
3,Q('PWF'),1.037175
2,Q('OSF'),1.030903
10,Q('Type'),1.011599
8,Q('Tool wear [min]'),1.006448
7,Q('TWF'),1.006107


### 3.2.2 Get permutation importance

In [21]:
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.25, stratify=y_data)

In [22]:
dtc = DecisionTreeClassifier()
dtc.fit(x_train.values, y_train.values)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [23]:
fs.get_permutation_importance(dtc, x_valid, y_valid)

,feature,perm_importance
0,Type,-0.000160
1,Air temperature [K],0.006906
2,Process temperature [K],0.006052
3,Rotational speed [rpm],0.003421
4,Tool wear [min],0.000474
5,Torque [Nm],0.003876
6,HDF,0.008112
7,OSF,0.006551
8,PWF,0.003839
9,RNF,0.000017


* 위 결과를 바탕으로 우선 RNF값을 제거하고 성능을 평가합니다.
* 그 후 Type에 대해서도 제거를 수행해봅니다.

* 모든 데이터 사용시

In [24]:
# Accuracy: 0.9960052774316499, f1_score: 0.8593548387096774, Roc-Auc: 0.8878698655479385
# Accuracy: 0.9960052774316499, f1_score: 0.858257477243173, Roc-Auc: 0.8835533699108423
# Accuracy: 0.9965916587260867, f1_score: 0.8824273072060682, Roc-Auc: 0.9055905397260844
# Accuracy: 0.9957120867844316, f1_score: 0.845442536327609, Roc-Auc: 0.8719626985611462
# Accuracy: 0.9958218801539307, f1_score: 0.8523316062176166, Roc-Auc: 0.8831892340605477
# array([0.99602724, 0.85956275, 0.88643314])

In [25]:
x_data.drop('RNF', axis=1, inplace=True)
x_test.drop('RNF', axis=1, inplace=True)

* RNF 제거 시 스코어

In [26]:
# Accuracy: 0.9960052774316499, f1_score: 0.8597168597168596, Roc-Auc: 0.8890167495931356
# Accuracy: 0.9960419262625523, f1_score: 0.859375, Roc-Auc: 0.8835719877243662
# Accuracy: 0.9966649563878912, f1_score: 0.883780332056194, Roc-Auc: 0.9021952567006809
# Accuracy: 0.9956021402917247, f1_score: 0.841688654353562, Roc-Auc: 0.8707626722364238
# Accuracy: 0.9958218801539307, f1_score: 0.8523316062176166, Roc-Auc: 0.8831892340605477
# array([0.99602724, 0.85937849, 0.88574718])

In [27]:
x_data.drop('Type', axis=1, inplace=True)
x_test.drop('Type', axis=1, inplace=True)

* 추가로 Type 제거 시 스코어

In [28]:
# Accuracy: 0.9960419262625523, f1_score: 0.8608247422680411, Roc-Auc: 0.8890353667134395
# Accuracy: 0.9960785750934545, f1_score: 0.8601307189542484, Roc-Auc: 0.8824464326537399
# Accuracy: 0.9966649563878912, f1_score: 0.8834827144686299, Roc-Auc: 0.9010510838165307
# Accuracy: 0.9955288426299201, f1_score: 0.8390501319261213, Roc-Auc: 0.8695812637252253
# Accuracy: 0.995968480850284, f1_score: 0.8567708333333334, Roc-Auc: 0.8832637053146436
# array([0.99605656, 0.86005183, 0.88507557])

# 4. Modeling

## 4.1 LogisticRegression

In [29]:
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.25, stratify=y_data)

In [30]:
lr = LogisticRegression()
lr.fit(x_train.values, y_train.values)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [31]:
acc_score = lr.score(x_train.values, y_train)
acc_score

0.9962177852053831

In [32]:
f1score = f1_score(y_train, lr.predict(x_train.values))
f1score

0.8644483362521891

In [33]:
acc_score = lr.score(x_valid.values, y_valid)
acc_score

0.9959247097455143

In [34]:
f1score = f1_score(y_valid, lr.predict(x_valid.values))
f1score

0.8522848034006377

## 4.2 XGBoost

In [36]:
# Define the objective function for Optuna
def objective(trial):
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'tree_method': 'gpu_hist',  # GPU 가속 사용
        'gpu_id': 0,  # 사용할 GPU 장치 ID 설정 (0 또는 다른 GPU ID)
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    valid_score = []

    for train_index, test_index in skf.split(x_data, y_data):
        x_train, x_valid = x_data.iloc[train_index], x_data.iloc[test_index]
        y_train, y_valid = y_data.iloc[train_index], y_data.iloc[test_index]

        xgb = XGBClassifier(**params)
        # Early stopping 설정
        xgb.fit(x_train.values, y_train.values, verbose=0, eval_set=[(x_valid.values, y_valid.values)],
                early_stopping_rounds=10)

        acc_score = xgb.score(x_valid, y_valid)
        predict_val = xgb.predict(x_valid)
        f1score = f1_score(y_valid, predict_val)
        roc_auc = roc_auc_score(y_valid, predict_val)
        valid_score.append((acc_score, f1score, roc_auc))

    # Calculate and report the mean F1-score for this trial
    mean_f1_score = sum([score[1] for score in valid_score]) / len(valid_score)
    trial.report(mean_f1_score, step=trial.number)

    return mean_f1_score

In [37]:
# Create an Optuna study and optimize
study = optuna.create_study(direction='maximize')  # Maximize the F1-score
study.optimize(objective, n_trials=50)  # Number of trials can be adjusted

# Get the best parameters
best_params = study.best_params
print(f"Best Parameters: {best_params}")

# Get the best F1-score achieved during the optimization
best_f1_score = study.best_value
print(f"Best F1-Score: {best_f1_score}")

[I 2023-09-22 15:05:49,308] A new study created in memory with name: no-name-0e8c3106-c795-4755-bbc4-2634c8ba16c7
[I 2023-09-22 15:06:01,414] Trial 0 finished with value: 0.8606431963526007 and parameters: {'n_estimators': 492, 'max_depth': 17, 'learning_rate': 0.06632124274739234, 'subsample': 0.5726541539816163, 'colsample_bytree': 0.7363022189820154}. Best is trial 0 with value: 0.8606431963526007.
[I 2023-09-22 15:06:26,536] Trial 1 finished with value: 0.8624585670942526 and parameters: {'n_estimators': 761, 'max_depth': 8, 'learning_rate': 0.01710992525291977, 'subsample': 0.8545136758613967, 'colsample_bytree': 0.8646143849192524}. Best is trial 1 with value: 0.8624585670942526.
[I 2023-09-22 15:06:45,232] Trial 2 finished with value: 0.8600123549731066 and parameters: {'n_estimators': 399, 'max_depth': 20, 'learning_rate': 0.0361502483262547, 'subsample': 0.8922688607022589, 'colsample_bytree': 0.913892955135641}. Best is trial 1 with value: 0.8624585670942526.
[I 2023-09-22 15

[I 2023-09-22 15:21:40,580] Trial 28 finished with value: 0.862770544775411 and parameters: {'n_estimators': 900, 'max_depth': 3, 'learning_rate': 0.01484599338033089, 'subsample': 0.7459229095123954, 'colsample_bytree': 0.5505681952657433}. Best is trial 10 with value: 0.8630436304641831.
[I 2023-09-22 15:22:22,311] Trial 29 finished with value: 0.8580339041980176 and parameters: {'n_estimators': 473, 'max_depth': 17, 'learning_rate': 0.004984577321497195, 'subsample': 0.6142242997530368, 'colsample_bytree': 0.579363706350672}. Best is trial 10 with value: 0.8630436304641831.
[I 2023-09-22 15:22:40,813] Trial 30 finished with value: 0.8617611869143487 and parameters: {'n_estimators': 696, 'max_depth': 20, 'learning_rate': 0.05239872116221006, 'subsample': 0.6909048265283438, 'colsample_bytree': 0.5002411323032582}. Best is trial 10 with value: 0.8630436304641831.
[I 2023-09-22 15:23:15,589] Trial 31 finished with value: 0.862770544775411 and parameters: {'n_estimators': 910, 'max_dept

Best Parameters: {'n_estimators': 988, 'max_depth': 9, 'learning_rate': 0.021746182242880132, 'subsample': 0.7788724729574976, 'colsample_bytree': 0.7822599132555701}
Best F1-Score: 0.8630436304641831


### 4.2.1 Create submission.csv

In [70]:
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.25, stratify=y_data)

In [71]:
xgb = XGBClassifier(**study.best_params)
xgb.fit(x_train.values, y_train.values, verbose=2)

[17:44:33] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_hidden_layers", "num_units" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.016976047514993316, max_bin=256,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=6, max_leaves=0,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_hidden_layers=3,
              num_parallel_tree=1, num_units=240, objective='binary:logistic',
              predictor='auto', ...)

In [72]:
prediction = xgb.predict_proba(x_test)[:,1]
len(prediction)

90954

In [73]:
result = test_df.copy()
result = result.drop(result.columns.difference(['id']), axis=1)
result['Machine failure'] = prediction
result

,id,Machine failure
0,136429,0.093477
1,136430,0.093477
2,136431,0.093477
3,136432,0.093477
4,136433,0.093477
...,...,...
90949,227378,0.093477
90950,227379,0.093477
90951,227380,0.093477
90952,227381,0.093477


In [74]:
result.to_csv('../data/submission.csv', index = False)

## 4.3 TensorFlow

In [64]:
# 특정 epoch 마다 출력할 Callback 정의
class VerboseEpochs(Callback):
    def __init__(self, verbose_epoch_interval):
        super(VerboseEpochs, self).__init__()
        self.verbose_epoch_interval = verbose_epoch_interval

    def on_epoch_end(self, epoch, logs=None):
        if epoch % self.verbose_epoch_interval == 0:
#             print(f"Epoch {epoch + 1}/{self.params['epochs']} - loss: {logs['loss']} - acc: {logs['binary_accuracy']}")
            print(f"Epoch {epoch + 1}/{self.params['epochs']} - val_loss: {logs['val_loss']} - val_acc: {logs['val_binary_accuracy']}")

# Callback 객체 생성
verbose_callback = VerboseEpochs(verbose_epoch_interval=10)  # 예를 들어, 매 5 epoch 마다 출력

In [65]:
# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=10, min_delta=0.0005, restore_best_weights=True)

In [66]:
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.25, stratify=y_data)

In [67]:
# 로그 설정
logging.basicConfig(filename='optuna_trials.log', level=logging.INFO)
    
def objective(trial):
    # 하이퍼파라미터 추천
    num_hidden_layers = trial.suggest_int('num_hidden_layers', 1, 3)
    num_units = trial.suggest_int('num_units', 32, 512, log=True)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    # 뉴런 수 감소 비율 튜닝 (예: 1.0에서 0.3 사이)
    neuron_reduction_rate = trial.suggest_float('neuron_reduction_rate', 0.3, 1)
    # 드롭아웃 비율 튜닝 (예: 0.0에서 0.5 사이)
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
    
    f1_scores = []  # 각 폴드의 F1 스코어를 저장할 리스트

    # 모델 구성
    model = Sequential()
    model.add(Dense(num_units, activation='relu', input_shape=(x_data.shape[1],)))

    for i in range(1, num_hidden_layers):
        num_units = int(num_units * neuron_reduction_rate)  # 뉴런 수 감소
        model.add(Dense(num_units, activation='relu'))

        model.add(Dropout(dropout_rate))  # 드롭아웃 레이어 추가 및 드롭아웃 비율 조정

        model.add(BatchNormalization())  # 배치 정규화 레이어 추가
    model.add(Dense(1, activation='sigmoid'))

    # 모델 컴파일
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=learning_rate), metrics=['binary_accuracy'])

    # 모델 훈련 및 평가
    history = model.fit(x_train, y_train, epochs=200, validation_data=(x_valid, y_valid),
                        callbacks=[early_stopping, verbose_callback], batch_size=128, verbose=0)

    # 검증 데이터에 대한 예측을 생성
    y_pred = model.predict(x_valid)
    y_pred = (y_pred > 0.5).astype(int)

    # F1 스코어 계산 및 저장
    val_f1_score = f1_score(y_valid, y_pred)
    f1_scores.append(val_f1_score)

    # 각 폴드의 F1 스코어 평균을 반환
    mean_f1_score = sum(f1_scores) / len(f1_scores)
    # 트라이얼 리포트
    trial.report(mean_f1_score, step=trial.number)
    logging.info(f"Trial {trial.number}: F1 Score = {mean_f1_score}")
    
    return mean_f1_score

In [68]:
study = optuna.create_study(direction='maximize')  # 최대화 문제로 설정
study.optimize(objective, n_trials=50)  # n_trials은 시도할 하이퍼파라미터 조합의 수

best_params = study.best_params
best_accuracy = study.best_value

print("Best Hyperparameters:", best_params)
print("Best Validation Accuracy:", best_accuracy)

[I 2023-09-22 20:14:07,814] A new study created in memory with name: no-name-23e0e816-4f52-441e-a405-51a0cec241aa


Epoch 1/200 - loss: 0.4922139048576355 - acc: 0.7870720624923706
Epoch 1/200 - val_loss: 0.12899523973464966 - val_acc: 0.9934326410293579
Epoch 11/200 - loss: 0.02534899301826954 - acc: 0.9956998229026794
Epoch 11/200 - val_loss: 0.02302493341267109 - val_acc: 0.9963644742965698
Epoch 21/200 - loss: 0.023372692987322807 - acc: 0.9958072900772095
Epoch 21/200 - val_loss: 0.021759988740086555 - val_acc: 0.9963937997817993
Epoch 31/200 - loss: 0.022776516154408455 - acc: 0.9959050416946411
Epoch 31/200 - val_loss: 0.021260298788547516 - val_acc: 0.9963351488113403


[I 2023-09-22 20:17:08,224] Trial 0 finished with value: 0.868559411146162 and parameters: {'num_hidden_layers': 3, 'num_units': 126, 'learning_rate': 0.00020378443930675307, 'neuron_reduction_rate': 0.6879792003596248, 'dropout_rate': 0.428504206225102}. Best is trial 0 with value: 0.868559411146162.


Epoch 1/200 - loss: 0.22744819521903992 - acc: 0.93666011095047
Epoch 1/200 - val_loss: 0.03651801496744156 - val_acc: 0.9960126876831055
Epoch 11/200 - loss: 0.022769341245293617 - acc: 0.9959148168563843
Epoch 11/200 - val_loss: 0.021128417924046516 - val_acc: 0.9964231252670288


[I 2023-09-22 20:18:04,220] Trial 1 finished with value: 0.8714733542319749 and parameters: {'num_hidden_layers': 2, 'num_units': 114, 'learning_rate': 0.00046592605998961063, 'neuron_reduction_rate': 0.37970693447989695, 'dropout_rate': 0.2627109233425965}. Best is trial 1 with value: 0.8714733542319749.


...

Epoch 1/200 - loss: 0.17051716148853302 - acc: 0.9542713761329651
Epoch 1/200 - val_loss: 0.05280723050236702 - val_acc: 0.9871290922164917
Epoch 11/200 - loss: 0.021881218999624252 - acc: 0.9960907101631165
Epoch 11/200 - val_loss: 0.020913569256663322 - val_acc: 0.9964231252670288
Epoch 21/200 - loss: 0.021312158554792404 - acc: 0.9961004853248596
Epoch 21/200 - val_loss: 0.020388340577483177 - val_acc: 0.9964231252670288


[I 2023-09-22 21:02:39,156] Trial 31 finished with value: 0.8721174004192872 and parameters: {'num_hidden_layers': 1, 'num_units': 53, 'learning_rate': 0.000801336977497399, 'neuron_reduction_rate': 0.6753695498256043, 'dropout_rate': 0.1894260897262743}. Best is trial 4 with value: 0.8732984293193717.


Epoch 1/200 - loss: 0.07278785854578018 - acc: 0.9873828291893005
Epoch 1/200 - val_loss: 0.02433069795370102 - val_acc: 0.9961006045341492


[W 2023-09-22 21:02:55,031] Trial 32 failed with parameters: {'num_hidden_layers': 1, 'num_units': 74, 'learning_rate': 0.0016878954588670848, 'neuron_reduction_rate': 0.4687466429639684, 'dropout_rate': 0.25495130503882535} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\ProgramData\miniconda3\envs\tfgpu\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\medici\AppData\Local\Temp\ipykernel_2856\2882808499.py", line 34, in objective
    callbacks=[early_stopping, verbose_callback], batch_size=128, verbose=0)
  File "C:\ProgramData\miniconda3\envs\tfgpu\lib\site-packages\tensorflow\python\keras\engine\training.py", line 1193, in fit
    tmp_logs = self.train_function(iterator)
  File "C:\ProgramData\miniconda3\envs\tfgpu\lib\site-packages\tensorflow\python\eager\def_function.py", line 885, in __call__
    result = self._call(*args, **kwds)
  File "C:\ProgramData\minic

KeyboardInterrupt: 

### 4.3.1 Create submission.csv

In [72]:
# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=10, min_delta=0.0001, restore_best_weights=True)

In [73]:
study.best_params

{'num_hidden_layers': 2,
 'num_units': 71,
 'learning_rate': 9.959937802862932e-05,
 'neuron_reduction_rate': 0.9624360355344637,
 'dropout_rate': 0.26850153604216426}

In [74]:
# Trial 4 finished with value: 0.8732984293193717 and parameters:
# {'num_hidden_layers': 2, 'num_units': 71, 'learning_rate': 9.959937802862932e-05,
# 'neuron_reduction_rate': 0.9624360355344637, 'dropout_rate': 0.26850153604216426}.
# Best is trial 4 with value: 0.8732984293193717.

In [75]:
num_hidden_layers = 2
num_units = 71
neuron_reduction_rate = 0.9624360355344637
dropout_rate = 0.26850153604216426
learning_rate = 9.959937802862932e-05

# 모델 구조 정의하기
model = Sequential()
model.add(Dense(num_units, activation='relu', input_shape=(x_data.shape[1],)))
for i in range(1, num_hidden_layers):
    num_units = int(num_units * neuron_reduction_rate)  # 뉴런 수 감소
    model.add(Dense(num_units, activation='relu'))
    model.add(Dropout(dropout_rate))  # 드롭아웃 레이어 추가 및 드롭아웃 비율 조정
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=learning_rate), metrics=['binary_accuracy'])

In [77]:
model.fit(x_train, y_train, epochs=200, validation_data=(x_valid, y_valid), callbacks=[early_stopping], batch_size=128)

Epoch 1/200
800/800 [==============================] - 3s 4ms/step - loss: 0.0302 - binary_accuracy: 0.9941 - val_loss: 0.0260 - val_binary_accuracy: 0.9948
Epoch 2/200
800/800 [==============================] - 3s 4ms/step - loss: 0.0274 - binary_accuracy: 0.9949 - val_loss: 0.0239 - val_binary_accuracy: 0.9957
Epoch 3/200
800/800 [==============================] - 3s 4ms/step - loss: 0.0258 - binary_accuracy: 0.9955 - val_loss: 0.0229 - val_binary_accuracy: 0.9961
Epoch 4/200
800/800 [==============================] - 3s 4ms/step - loss: 0.0250 - binary_accuracy: 0.9957 - val_loss: 0.0222 - val_binary_accuracy: 0.9962
Epoch 5/200
800/800 [==============================] - 3s 4ms/step - loss: 0.0241 - binary_accuracy: 0.9959 - val_loss: 0.0219 - val_binary_accuracy: 0.9962
Epoch 6/200
800/800 [==============================] - 3s 4ms/step - loss: 0.0236 - binary_accuracy: 0.9959 - val_loss: 0.0216 - val_binary_accuracy: 0.9962
Epoch 7/200
800/800 [==============================] - 3s 

Epoch 53/200
800/800 [==============================] - 3s 4ms/step - loss: 0.0213 - binary_accuracy: 0.9961 - val_loss: 0.0200 - val_binary_accuracy: 0.9964
Epoch 54/200
800/800 [==============================] - 3s 4ms/step - loss: 0.0216 - binary_accuracy: 0.9961 - val_loss: 0.0201 - val_binary_accuracy: 0.9964
Epoch 55/200
800/800 [==============================] - 3s 4ms/step - loss: 0.0212 - binary_accuracy: 0.9961 - val_loss: 0.0201 - val_binary_accuracy: 0.9964
Epoch 56/200
800/800 [==============================] - 3s 4ms/step - loss: 0.0213 - binary_accuracy: 0.9961 - val_loss: 0.0201 - val_binary_accuracy: 0.9964
Epoch 57/200
800/800 [==============================] - 3s 4ms/step - loss: 0.0212 - binary_accuracy: 0.9961 - val_loss: 0.0202 - val_binary_accuracy: 0.9964
Epoch 58/200
800/800 [==============================] - 3s 4ms/step - loss: 0.0211 - binary_accuracy: 0.9961 - val_loss: 0.0201 - val_binary_accuracy: 0.9964


In [78]:
predicted_probabilities = model.predict(x_valid)
predicted_classes = (predicted_probabilities > 0.5).astype(int)
f1score = f1_score(y_valid, predicted_classes)
f1score

0.8709338929695698

In [79]:
predicted_probabilities = model.predict(x_test)
len(predicted_probabilities)

90954

In [80]:
result = test_df.copy()
result = result.drop(result.columns.difference(['id']), axis=1)
result['Machine failure'] = predicted_probabilities
result

,id,Machine failure
0,136429,0.001108
1,136430,0.000649
2,136431,0.000780
3,136432,0.001739
4,136433,0.001461
...,...,...
90949,227378,0.001418
90950,227379,0.000335
90951,227380,0.003562
90952,227381,0.000601


In [81]:
result.to_csv('../data/submission.csv', index = False)